In [1]:
import os
import cv2
from tqdm import tqdm

In [2]:
dataset_path = "../datasets/beetle-tracking/Detection_data/"

In [3]:
##Get all .txt files
def _get_all_txt(folder_path):
    f = []
    idx = 0
    if os.path.exists(folder_path):
        for x in os.listdir(folder_path):
            name, ext = os.path.splitext(x)
            if ext == '.txt':
                #print(x)
                f.append(os.path.join(folder_path,x))
    else:
        print("%s does not exist!"%(folder_path))
    
    return f
    
#txt_files = _get_all_txt(dataset_path)

In [4]:
##Read description file and transfer the content to py dict.
def _read_box_desc(file_name):
    f = open(file_name,'r')
    result = dict()
    res = dict()
    cnt = 0
    result['fn'] = file_name
    vname, _ = os.path.splitext(file_name)
    result['name'] = os.path.split(vname)[-1]
    result['name'] = result['name'].replace(" ", "")
    result['vfn'] = "%s.avi"%vname[:-6]
    
    line = f.readline()
    while line:
        cnt = cnt+1
        line = eval("[%s]"%line)
        if len(line)>2:
            print("err", line)
            return result
        
        res[line[0]] = line[1]
        line = f.readline()
    
    f.close()
    result['res'] = res
    result['cnt'] = cnt
    return result

#r = _read_box_desc(txt_files[0])
#print(r['fn'])
#print(r['vfn'])
#print(r['cnt'])
#print(r['name'])
#print(r['res'])

In [5]:
## for debugging
def _pick_color(id):
    if id==0:
        cr = (0,0,200)
    elif id==1:
        cr = (0,200,0)
    elif id==2: 
        cr = (200,0,0)
    elif id==3: 
        cr = (0,100,100)
    elif id==4: 
        cr = (100,100, 0)
    else:
        cr = (255,255,255)
    
    return cr

In [7]:
## Read target video files and extract frames as image file according to the description file
def _capture_image_from_desc(desc, out_txt_name, out_img_folder, wBox=True):
    print(desc['vfn'], desc['name'])
    
    #input video path
    vf = desc['vfn']
    #output training_set name
    lname = desc['name']
    
    if not os.path.isfile(vf):
        print("wrong video file")
        return
    
    #output image folder path
    out_img_folder = os.path.join(out_img_folder, lname)
    if not os.path.isdir(out_img_folder):
        print("create out dir %s"%out_img_folder)
        os.makedirs(out_img_folder)
    
    #output training set index file
    if out_txt_name == '':
        out_txt_name = 'train.txt'
        out_txt_log_name = 'train_log.txt'
    else:
        out_txt_name = '%s.txt'%out_txt_name
        out_txt_log_name = '%s_log.txt'%out_txt_name
        
    out_txt_path = os.path.join(out_img_folder, out_txt_name)
    out_txt_log_path = os.path.join(out_img_folder, out_txt_log_name)
    print("output img\n%s \ntxt\n%s \nlog\n%s"%(out_img_folder,out_txt_path,out_txt_log_path))
    fw = open(out_txt_path,'w')
    fw_log = open(out_txt_log_path,'w')
    
    #read video files
    cap = cv2.VideoCapture(vf)
    timeout = 0
    while not cap.isOpened():
        #cap = cv2.VideoCapture(vf)
        cv2.waitKey(1000)
        print("Wait for the header")
        timeout = timeout +1
        if(timeout>10):
            break
    #print(cap.isOpened())
    total_frames = cap.get(7) ## CV_CAP_PROP_FRAME_COUNT
    print("total_frames %d"%total_frames)    
    
    res = desc['res']
    fcnt = 0
    
    
    for f_idx in tqdm(res.keys()):#go through specific frames and save it
        if f_idx>total_frames:
            print("parse error. incorrect frame idx %d-%d"%(f_idx,total_frames))
            return
        
        #print(f_idx, res[f_idx])
        fw_log.write("%s %s\n"%(f_idx,str(res[f_idx])))
        cap.set(1, f_idx) #CV_CAP_PROP_POS_FRAMES, frame_num
        _, frame = cap.read()
        fcnt = fcnt +1
        img_path = os.path.join(out_img_folder, "f%05d_%05d.jpg"%(fcnt, f_idx))
        #cv2.imwrite(img_path, frame)
        out_str = "%s"%(img_path)
        frame_box = frame
        TYPE_BUG = 0
        TYPE_UNKNOWN = 1
        for bug_id,left,right in res[f_idx]:
            #print(bug_id,left,right)
            cr = _pick_color(bug_id)
            cv2.rectangle(frame_box,left,right,cr,2)       
            cv2.putText(frame_box,str(bug_id),left,0,1,cr)
            out_str = "%s %d,%d,%d,%d,%d"%(out_str, left[0], left[1], right[0], right[1], TYPE_BUG)
        
        out_str = "%s\n"%out_str
        fw.write(out_str)
        if wBox:
            img_path_b = os.path.join(out_img_folder, "f%05d_%05d_box.jpg"%(fcnt, f_idx))
            cv2.imwrite(img_path_b, frame_box)
        
        #if fcnt>10:
        #    break
    
    fw.close()
    fw_log.close()
    cap.release()    
    cv2.destroyAllWindows()
    print("capture done %s"%vf)
    return 

#ff = _capture_image_from_desc(r, '', './ds_train/')


In [8]:
##main
txt_files = _get_all_txt(dataset_path)
for txt in txt_files:
    d = _read_box_desc(txt)
    _capture_image_from_desc(d, '', './ds_train2/',False)
    break

  0%|          | 0/300 [00:00<?, ?it/s]

../datasets/beetle-tracking/Detection_data/[CH03] 2017-08-12 21.15.00_x264.avi [CH03]2017-08-1221.15.00_x264_label
create out dir ./ds_train2/[CH03]2017-08-1221.15.00_x264_label
output img
./ds_train2/[CH03]2017-08-1221.15.00_x264_label 
txt
./ds_train2/[CH03]2017-08-1221.15.00_x264_label/train.txt 
log
./ds_train2/[CH03]2017-08-1221.15.00_x264_label/train_log.txt
total_frames 26968


100%|██████████| 300/300 [00:48<00:00,  6.39it/s]

capture done ../datasets/beetle-tracking/Detection_data/[CH03] 2017-08-12 21.15.00_x264.avi


In [24]:
txt_files = _get_all_txt(dataset_path)
for txt in txt_files:
    d = _read_box_desc(txt)
    res = d['res']
    for f_idx in tqdm(res.keys()):
        print(f_idx)
        pass
    
    break

100%|██████████| 300/300 [00:00<00:00, 25698.82it/s]

1
78
395
1407
1615
2027
2057
2257
2357
2430
3179
3404
3715
4098
4375
4544
5245
5348
5476
5623
5628
5759
6063
6076
6127
6171
6215
6322
6458
6482
6664
6753
6952
6966
6984
7011
7034
7129
7204
7274
7345
7376
7650
7791
7986
8012
8015
8124
8227
8228
8251
8263
8334
8429
8578
8627
8651
8721
8743
8996
9016
9173
9220
9323
9408
9457
9551
9604
9621
9644
9722
9840
9907
9915
9927
9954
9985
10015
10067
10079
10115
10128
10176
10183
10274
10356
10403
10439
10478
10561
10594
10696
10794
10840
10891
10920
10925
10999
11008
11054
11070
11082
11105
11135
11178
11191
11200
11229
11294
11310
11329
11348
11378
11412
11421
11436
11474
11500
11502
11610
11638
11691
11798
11823
11853
11892
11899
11941
11976
11999
12141
12298
12325
12369
12492
12536
12562
12611
12623
12669
12759
12871
12914
12917
12943
12999
13143
13174
13259
13349
13404
13427
13447
13520
13526
13563
13678
13680
13792
13862
13867
13879
14008
14014
14062
14099
14130
14165
14191
14202
14265
14407
14456
14499
14538
14604
14611
14917
15066
15124
151